
<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
Artificial Intelligence <br>
<font color=2565AE size=5>
Computer Engineering Department <br>
Spring 2024<br>
<font color=3C99D size=5>
Practical Assignment - Minimax <br>
<font color=696880 size=4>
Ali Aghayari


____

# P0 : Game explanation and environment setup (0 points)

In this Jupyter notebook, we aim to develop the AI logic for the game **Connect4**. Players take turns dropping pieces into a grid, and the first player to align four consecutive pieces vertically, horizontally, or diagonally wins. The focus is on creating an intelligent AI opponent using the minimax algorithm with alpha-beta pruning to deliver a challenging gameplay experience.

<span style="color: red;">However, there’s an added twist: after either player drops a piece, there is a 12.5% chance that the entire board will rotate 90 degrees clockwise, changing the direction of gravity as well! Keep this rule in mind when implementing your heuristic, as your AI will face a tough challenge ahead.</span>

<br>

Note: The winning condition will be checked after applying the rotation. If both sides have winning conditions, the player who made the last move will lose.

<br>
Note: For full clarification, the following occurs:
- Some players make their moves.
- Rotations happens. (12.5%)
- New gravity is applied to every piece on the board. (12.5%)
- The winning condition is then checked.

<br>
Rules: Do not modify the code cells that don’t have TODO comments, except for those that are explicitly mentioned as okay to change.

First, we will define some constants to make the code cleaner, more organized, and to set up the game environment.

In [32]:
# Game Constants
ROW_COUNT = 7
COLUMN_COUNT = 7
WINDOW_LENGTH = 4
EMPTY = 0

# Players and Pieces
PLAYER = 0
AI = 1
EMPTY_PIECE = 0
PLAYER_PIECE = 1
AI_PIECE = 2

# Colors (RGB values) - you can change the colors to your liking
FG_COLOR = (0, 0, 255)
BG_COLOR = (0, 0, 0)
P1_COLOR = (255, 0, 0)
P2_COLOR = (0, 255, 0)

# Pygame Constants
SQUARESIZE = 80
RADIUS = SQUARESIZE // 2 - 5
SCREEN_WIDTH = COLUMN_COUNT * SQUARESIZE
SCREEN_HEIGHT = (ROW_COUNT + 1) * SQUARESIZE

These libraries are sufficient to complete this task, but feel free to add any additional imports you may need.

In [33]:
!pip install pygame

import numpy as np
import random
import pygame
import math
import time

# Additional libraries can be imported here if needed



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# P1 : Util functions (20 points)

We need to initialize the game board as an empty 2D array with dimensions of ROW_COUNT by COLUMN_COUNT.

In [34]:
def create_board():
    """
    Initializes an empty game board with dimensions ROW_COUNT x COLUMN_COUNT.
    Each cell is set to EMPTY, representing an unoccupied position.
    
    Returns:
        numpy.ndarray: A 2D array representing the game board.
    """
    board = np.zeros((ROW_COUNT, COLUMN_COUNT), dtype=int)  # Create a 2D array filled with zeros
    return board


Fill in the code to find the valid columns where a piece can be dropped.

In [56]:
def is_valid_location(board, col):
    """
    Checks if the specified column has space for another piece.
    
    Args:
        board (numpy.ndarray): The game board.
        col (int): The column to check.
    
    Returns:
        bool: True if the top of the column is empty, indicating space is available.
    """
    return board[0][col] == EMPTY  # Check if the top row of the column is empty


In [57]:
def get_valid_locations(board):
    """
    Finds all columns that have at least one empty cell.
    
    Args:
        board (numpy.ndarray): The game board.
    
    Returns:
        list: A list of valid column indices where pieces can be dropped.
    """
    valid_locations = []
    for col in range(COLUMN_COUNT):
        if is_valid_location(board, col):  # Check if column has an open spot
            valid_locations.append(col)
    return valid_locations


Fill in the code to find the valid row for the given column where a piece can be dropped.

In [58]:
def get_next_open_row(board, col):
    """
    Finds the next open row in the given column where a piece can be dropped.
    
    Args:
        board (numpy.ndarray): The current game board.
        col (int): The column to check.
    
    Returns:
        int: The index of the next open row in the specified column, or -1 if the column is full.
    """
    for row in range(ROW_COUNT - 1, -1, -1):  # Start from the bottom row and go upwards
        if board[row][col] == EMPTY:          # Check if the cell is empty
            return row
    return -1  # Return -1 if the column is full


Fill in the code to drop a piece in the specified column of the board.

In [59]:
def drop_piece(board, col, piece):
    """
    Drops a piece into the specified column on the board.
    
    Args:
        board (numpy.ndarray): The current game board.
        col (int): The column where the piece will be dropped.
        piece (int): The piece type (e.g., PLAYER_PIECE or AI_PIECE).
    
    Returns:
        None
    """
    # Find the next open row in the specified column
    row = get_next_open_row(board, col)
    
    if row != -1:  # Check if the column is not full
        board[row][col] = piece  # Place the piece in the board


Fill in the code to check if the specified piece has won the game.

In [60]:
def winning_move(board, piece):
    """
    Checks if the specified piece has achieved a winning move.
    A winning move consists of four consecutive pieces either horizontally, vertically, or diagonally.
    
    Args:
        board (numpy.ndarray): The current game board.
        piece (int): The piece type to check (e.g., PLAYER_PIECE or AI_PIECE).
    
    Returns:
        bool: True if the specified piece has won, False otherwise.
    """

    # Horizontal win
    for row in range(ROW_COUNT):
        for col in range(COLUMN_COUNT - 3):
            if np.all(board[row, col:col+4] == piece):
                return True

    # Vertical win
    for col in range(COLUMN_COUNT):
        for row in range(ROW_COUNT - 3):
            if np.all([board[row+i][col] == piece for i in range(4)]):
                return True

    # Positive diagonal win (\)
    for row in range(ROW_COUNT - 3):
        for col in range(COLUMN_COUNT - 3):
            if np.all([board[row+i][col+i] == piece for i in range(4)]):
                return True

    # Negative diagonal win (/)
    for row in range(3, ROW_COUNT):
        for col in range(COLUMN_COUNT - 3):
            if np.all([board[row-i][col+i] == piece for i in range(4)]):
                return True

    return False


# P2 : Scoring function and Minimax implementation (50 points)

Fill in the code to score the current situation for the player. Hint: You can divide the board into separate windows, score each window individually for the given piece, and sum the scores to obtain the total board score for that piece.

In [47]:
def evaluate_window(window, piece):
    """
    Scores a window (subset of the board) for the specified piece.
    The scoring function evaluates the number of pieces and empty spaces in the window.
    
    Args:
        window (numpy.ndarray): A subset of the board (e.g., 4 consecutive cells).
        piece (int): The piece type to evaluate (e.g., PLAYER_PIECE or AI_PIECE).
    
    Returns:
        int: The score for the given window.
    """
    score = 0
    opponent_piece = PLAYER_PIECE if piece == AI_PIECE else AI_PIECE

    piece_count = np.count_nonzero(window == piece)
    empty_count = np.count_nonzero(window == EMPTY)
    opponent_count = np.count_nonzero(window == opponent_piece)

    if piece_count == 4:  # Winning condition
        score += 100
    elif piece_count == 3 and empty_count == 1:  # Strong position
        score += 10
    elif piece_count == 2 and empty_count == 2:  # Decent position
        score += 5

    if opponent_count == 3 and empty_count == 1:  # Block opponent's win
        score -= 80

    return score


In [61]:
def score_position(board, piece):
    """
    Scores the entire board for the given piece by evaluating all possible windows.
    
    Args:
        board (numpy.ndarray): The current game board.
        piece (int): The piece type to evaluate (e.g., PLAYER_PIECE or AI_PIECE).
    
    Returns:
        int: The aggregated score for the board.
    """
    score = 0
    center_column = COLUMN_COUNT // 2  # Center column index
    opponent_piece = PLAYER_PIECE if piece == AI_PIECE else AI_PIECE

    # Score center column (prefer central positions for strategic advantage)
    center_array = [board[row][center_column] for row in range(ROW_COUNT)]
    center_count = center_array.count(piece)
    score += center_count * 6  # Heuristic: prefer pieces in the center

    # Horizontal scoring
    for row in range(ROW_COUNT):
        for col in range(COLUMN_COUNT - 3):  # Ensure there's space for a window
            window = board[row, col:col + 4]
            score += evaluate_window(window, piece)

    # Vertical scoring
    for col in range(COLUMN_COUNT):
        for row in range(ROW_COUNT - 3):  # Ensure there's space for a window
            window = [board[row + i][col] for i in range(4)]
            score += evaluate_window(window, piece)

    # Positively sloped diagonal scoring (\)
    for row in range(ROW_COUNT - 3):
        for col in range(COLUMN_COUNT - 3):  # Ensure there's space for a window
            window = [board[row + i][col + i] for i in range(4)]
            score += evaluate_window(window, piece)

    # Negatively sloped diagonal scoring (/)
    for row in range(3, ROW_COUNT):
        for col in range(COLUMN_COUNT - 3):  # Ensure there's space for a window
            window = [board[row - i][col + i] for i in range(4)]
            score += evaluate_window(window, piece)

    return score


Fill in the code to implement the minimax algorithm with alpha beta pruning using the utility function provided.

In [62]:
def minimax(board, depth, alpha, beta, maximizingPlayer):
    """
    Implements the minimax algorithm with alpha-beta pruning to find the best move.
    
    Args:
        board (numpy.ndarray): The current game board.
        depth (int): The maximum depth of the recursion.
        alpha (float): The alpha value for pruning (initially -infinity).
        beta (float): The beta value for pruning (initially infinity).
        maximizingPlayer (bool): True if the current move is for the maximizing player (AI), False for the minimizing player (human).
    
    Returns:
        tuple: (best_column, value) where `best_column` is the optimal column to play in, and `value` is the minimax evaluation score.
    """
    valid_locations = get_valid_locations(board)
    is_terminal = (
    winning_move(board, PLAYER_PIECE) or 
    winning_move(board, AI_PIECE) or 
    len(get_valid_locations(board)) == 0 or 
    depth == 0
    )

    if is_terminal:
        if winning_move(board, AI_PIECE):  # AI wins
            return (None, 1000000)
        elif winning_move(board, PLAYER_PIECE):  # Player wins
            return (None, -1000000)
        elif len(valid_locations) == 0:  # Draw
            return (None, 0)
        else:  # Depth reached
            return (None, score_position(board, AI_PIECE))

    if maximizingPlayer:
        value = -float('inf')
        best_column = random.choice(valid_locations)  # Choose a random column as a fallback
        for col in valid_locations:
            temp_board = board.copy()
            drop_piece(temp_board, col, AI_PIECE)
            new_score = minimax(temp_board, depth - 1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                best_column = col
            alpha = max(alpha, value)
            if alpha >= beta:  # Prune
                break
        return best_column, value

    else:  # Minimizing player
        value = float('inf')
        best_column = random.choice(valid_locations)  # Choose a random column as a fallback
        for col in valid_locations:
            temp_board = board.copy()
            drop_piece(temp_board, col, PLAYER_PIECE)
            new_score = minimax(temp_board, depth - 1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                best_column = col
            beta = min(beta, value)
            if alpha >= beta:  # Prune
                break
        return best_column, value


# P3 : PVE (0 points)

Run this code to test your AI's performance. It is recommended to execute this part locally.

In [65]:
pygame.init()
width = COLUMN_COUNT * SQUARESIZE
height = (ROW_COUNT + 1) * SQUARESIZE
size = (width, height)
screen = pygame.display.set_mode(size)
myfont = pygame.font.SysFont("monospace", 75)

def rotate_board(board):
    rotated_board = np.rot90(board)
    for col in range(rotated_board.shape[1]):
        column = rotated_board[:, col]
        non_empty = column[column != EMPTY]
        empty_count = ROW_COUNT - len(non_empty)
        rotated_board[:, col] = np.concatenate((non_empty, np.zeros(empty_count, dtype=int)))
    return rotated_board


def draw_board(board):
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            pygame.draw.rect(screen, FG_COLOR, (c * SQUARESIZE, r * SQUARESIZE + SQUARESIZE, SQUARESIZE, SQUARESIZE))
            pygame.draw.circle(screen, BG_COLOR, (
                int(c * SQUARESIZE + SQUARESIZE / 2), int(r * SQUARESIZE + SQUARESIZE + SQUARESIZE / 2)), RADIUS)

    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            if board[r][c] == PLAYER_PIECE:
                pygame.draw.circle(screen, P1_COLOR, (
                    int(c * SQUARESIZE + SQUARESIZE / 2), height - int(r * SQUARESIZE + SQUARESIZE / 2)), RADIUS)
            elif board[r][c] == AI_PIECE:
                pygame.draw.circle(screen, P2_COLOR, (
                    int(c * SQUARESIZE + SQUARESIZE / 2), height - int(r * SQUARESIZE + SQUARESIZE / 2)), RADIUS)
    pygame.display.update()

def run_game():
    board = create_board()
    draw_board(board)
    game_over = False
    turn = random.choice([PLAYER, AI])  # Randomly select who goes first

    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

            # Player's turn
            if turn == PLAYER:
                if event.type == pygame.MOUSEMOTION:
                    pygame.draw.rect(screen, BG_COLOR, (0, 0, width, SQUARESIZE))
                    posx = event.pos[0]
                    pygame.draw.circle(screen, P1_COLOR, (posx, int(SQUARESIZE / 2)), RADIUS)
                    pygame.display.update()

                if event.type == pygame.MOUSEBUTTONDOWN:
                    pygame.draw.rect(screen, BG_COLOR, (0, 0, width, SQUARESIZE))
                    posx = event.pos[0]
                    col = int(math.floor(posx / SQUARESIZE))

                    if is_valid_location(board, col):
                        drop_piece(board, col, PLAYER_PIECE)
                        if random.random() <= 0.125:
                            board = rotate_board(board)
                            draw_board(board)
                            pygame.time.wait(1500)

                        if winning_move(board, PLAYER_PIECE):
                            label = myfont.render("Player 1 wins!!", 1, P1_COLOR)
                            print("Player 1 wins!!")
                            screen.blit(label, (40, 10))
                            game_over = True
                        elif winning_move(board, AI_PIECE):
                            label = myfont.render("Player 2 wins!!", 1, P2_COLOR)
                            print("Player 2 wins!!")
                            screen.blit(label, (40, 10))
                            game_over = True

                        turn = AI  # Switch turn to AI
                        draw_board(board)

                     # AI's turn
        if turn == AI and not game_over:
            col, minimax_score = minimax(board, 5, -math.inf, math.inf, True)

            if col is not None and is_valid_location(board, col):
                drop_piece(board, col, AI_PIECE)
                
                if random.random() <= 0.125:
                    board = rotate_board(board)
                    draw_board(board)
                    pygame.time.wait(1500)

                if winning_move(board, AI_PIECE):
                    label = myfont.render("Player 2 wins!!", 1, P2_COLOR)
                    print("Player 2 wins!!")
                    screen.blit(label, (40, 10))
                    game_over = True
                elif winning_move(board, PLAYER_PIECE):
                    label = myfont.render("Player 1 wins!!", 1, P1_COLOR)
                    print("Player 1 wins!!")
                    screen.blit(label, (40, 10))
                    game_over = True

                turn = PLAYER  # Switch turn back to Player
                draw_board(board)

        if game_over:
            pygame.time.wait(3000)
            pygame.quit()
            return



run_game()

# P4 : EVE (30 points)

In this section, we will simulate an AI battle where your AI heuristic should outperform our provided heuristic. Don’t worry; the opposing AI is not optimal, but if your scoring approach is inadequate, you may lose some credit from P2. Your AI should demonstrate a significant advantage, meaning it should consistently beat our AI on average, regardless of whether it plays as the first or second player. Please note that your search tree should can't have a higher depth than our heuristic.

Implement the minimax algorithm similar to your main minimax function. However, do not modify the tester_evaluate_window and tester_score_position functions. Remember to utilize tester_score_position within tester_minimax!

In [69]:
def tester_evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER_PIECE if piece == AI_PIECE else AI_PIECE
    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2
    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4
    return score


def tester_score_position(board, piece):
    score = 0
    center_array = [int(i) for i in list(board[:, COLUMN_COUNT // 2])]
    score += center_array.count(piece) * 3

    for r in range(ROW_COUNT):
        row_array = [int(i) for i in list(board[r, :])]
        for c in range(COLUMN_COUNT - 3):
            window = row_array[c:c + WINDOW_LENGTH]
            score += tester_evaluate_window(window, piece)

    for c in range(COLUMN_COUNT):
        col_array = [int(i) for i in list(board[:, c])]
        for r in range(ROW_COUNT - 3):
            window = col_array[r:r + WINDOW_LENGTH]
            score += tester_evaluate_window(window, piece)

    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r + i][c + i] for i in range(WINDOW_LENGTH)]
            score += tester_evaluate_window(window, piece)

    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r + 3 - i][c + i] for i in range(WINDOW_LENGTH)]
            score += tester_evaluate_window(window, piece)


def tester_minimax(board, depth, alpha, beta, maximizingPlayer):
    """
    Minimax algorithm with alpha-beta pruning for AI testing.
    
    Args:
        board (numpy.ndarray): The current game board.
        depth (int): The maximum depth of the recursion.
        alpha (float): The alpha value for pruning (initially -infinity).
        beta (float): The beta value for pruning (initially infinity).
        maximizingPlayer (bool): True if the current move is for the maximizing player (AI), False for the minimizing player (PLAYER).
    
    Returns:
        tuple: (best_column, value) where `best_column` is the optimal column to play in, and `value` is the minimax evaluation score.
    """
    valid_locations = get_valid_locations(board)
    is_terminal = winning_move(board, PLAYER_PIECE) or winning_move(board, AI_PIECE) or len(valid_locations) == 0 or depth == 0

    # Terminal condition: check for a win, loss, draw, or depth limit
    if is_terminal:
        if winning_move(board, AI_PIECE):  # AI wins
            return (None, 1000000)
        elif winning_move(board, PLAYER_PIECE):  # PLAYER wins
            return (None, -1000000)
        elif len(valid_locations) == 0:  # Draw
            return (None, 0)
        else:  # Depth reached
            return (None, tester_score_position(board, AI_PIECE))

    if maximizingPlayer:
        value = -float('inf')
        best_column = valid_locations[0]  # Default to the first valid column in case no better column is found
        for col in valid_locations:
            temp_board = board.copy()
            drop_piece(temp_board, col, AI_PIECE)
            new_score = tester_minimax(temp_board, depth - 1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                best_column = col
            alpha = max(alpha, value)
            if alpha >= beta:  # Alpha-beta pruning
                break
        return best_column, value

    else:  # Minimizing player (the tester's heuristic AI)
        value = float('inf')
        best_column = valid_locations[0]  # Default to the first valid column in case no better column is found
        for col in valid_locations:
            temp_board = board.copy()
            drop_piece(temp_board, col, PLAYER_PIECE)
            new_score = tester_minimax(temp_board, depth - 1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                best_column = col
            beta = min(beta, value)
            if alpha >= beta:  # Alpha-beta pruning
                break
        return best_column, value



In [70]:
def simulate_game():
    board = create_board()
    starting_turn = turn = random.choice([0, 1])

    game_over = False
    while not game_over:
        if turn == PLAYER:
            col, minimax_score = tester_minimax(board, 4, -math.inf, math.inf, True)
            if is_valid_location(board, col):
                drop_piece(board, col, PLAYER_PIECE)
                if random.random() <= 0.125:
                    board = rotate_board(board)

                if winning_move(board, AI_PIECE):
                    return 1, starting_turn

                elif winning_move(board, PLAYER_PIECE):
                    return 0, starting_turn
                turn += 1
                turn = turn % 2

        if turn == AI and not game_over:
            col, minimax_score = minimax(board, 4, -math.inf, math.inf, True)
            if is_valid_location(board, col):
                drop_piece(board, col, AI_PIECE)
                if random.random() <= 0.125:
                    board = rotate_board(board)
                if winning_move(board, PLAYER_PIECE):
                    return 0, starting_turn
                elif winning_move(board, AI_PIECE):
                    return 1, starting_turn

                turn += 1
                turn = turn % 2

Run this tester. You need to win at least 65% of the games to pass. 
<br>
The code execution should take less than 10 minutes to complete. If it exceeds this time, performance optimization might be necessary.

In [71]:
wins = 0
tests = 100
throw = 0
disadvantage = 0

start_time = time.time()

for i in range(tests):
    result = simulate_game()
    if result[1] == PLAYER and result[0] == 0: disadvantage += 1
    if result[1] == AI and result[0] == 0: throw += 1
    wins += result[0]

end_time = time.time()

print(f"you won {wins / tests}% of games")
print(f"you throw {throw / tests}% of games")
print(f"you lost logically {disadvantage / tests}% of games")
print(f"Code execution time: {end_time - start_time:.4f} seconds")

TypeError: '<' not supported between instances of 'NoneType' and 'float'